論文<br>
https://arxiv.org/abs/2206.07038<br>
<br>
GitHub<br>
https://github.com/TencentARC/AnimeSR<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/AnimeSR_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/TencentARC/AnimeSR.git

%cd /content/AnimeSR
# Commits on Feb 10, 2023
!git checkout 2c1a74231632ec70d485d09f45bce8afed688193

## ライブラリのインストール

In [ ]:
%cd /content/AnimeSR

# Install dependent packages
!pip install -r requirements.txt
!pip install --upgrade gdown
!pip install moviepy==0.2.3.5 imageio==2.4.1
!pip install yt-dlp

# Install AnimeSR
!python setup.py develop

## ライブラリのインポート

In [ ]:
from yt_dlp import YoutubeDL
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

# 学習済みモデルのセットアップ

In [ ]:
!gdown --id 1wpBXS5PIKDAC8IvMuCGNnrrA_fMTzF9P \
        -O ./weights/AnimeSR_v1-PaperModel.pth

!gdown --id 1E_qsFqlIre-fMUSSLADQboBssNIWSqmm \
        -O ./weights/AnimeSR_v2.pth

# テスト動画のセットアップ

In [ ]:
video_url = 'https://www.youtube.com/watch?v=nlpSopy0d-U' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  70#@param {type:"integer"}
end_sec =  75#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [ ]:
%cd /content/AnimeSR

!mkdir -p test_video

download_resolution = 720
full_video_path = './test_video/full_video.mp4'
input_clip_path = './test_video/clip_video.mp4'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

In [ ]:
# 動画の確認
clip = VideoFileClip(input_clip_path)
clip = resize(clip, height=420)
clip.ipython_display()

# Inference

In [ ]:
%cd /content/AnimeSR

'''
Usage:
  -i --input           Input video path or extracted frames folder
  -n --model_name      AnimeSR model name. Default: AnimeSR_v2, can also be AnimeSR_v1-PaperModel
  -s --outscale        The netscale is x4, but you can achieve arbitrary output scale (e.g., x2 or x1) with the argument outscale.
                       The program will further perform cheap resize operation after the AnimeSR output. Default: 4
  -o -output           Output root. Default: results
  -expname             Identify the name of your current inference. The outputs will be saved in $output/$expname
  -fps                 The fps of the (possible) saved videos. Default: None
  -extract_frame_first If input is a video, you can still extract the frames first, other wise AnimeSR will read from stream
  -num_process_per_gpu Since the slow I/O speed will make GPU utilization not high enough, so as long as the
                       video memory is sufficient, we recommend placing multiple processes on one GPU to increase the utilization of each GPU.
                       The total process will be number_process_per_gpu * num_gpu
  -suffix              You can add a suffix string to the sr video name, for example, 1gpu3processx2 which means the SR video is generated with one GPU and three process and the outscale is x2
  -half                Use half precision for inference, it won't make big impact on the visual results
'''

!CUDA_VISIBLE_DEVICES=0 python scripts/inference_animesr_video.py \
  -i ./test_video/clip_video.mp4 \
  -n AnimeSR_v2 \
  -s 4 \
  --expname animesr_v2 \
  --num_process_per_gpu 1 \
  --suffix 1gpu1process


In [ ]:
# 動画の確認
clip = VideoFileClip("/content/AnimeSR/results/animesr_v2/videos/clip_video/clip_video_1gpu1process.mp4")
clip = resize(clip, height=420)
clip.ipython_display()